<a href="https://colab.research.google.com/github/Sudaganivamsi/Vamsi_FMML-MODULES/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?



ChatGPT Auto
You said:
Does averaging the validation accuracy across multiple splits give more consistent results?
ChatGPT said:
ChatGPT
Yes, averaging the validation accuracy across multiple splits (e.g., in cross-validation) generally gives more consistent and reliable results. This approach helps to:
Reduce Variance: By evaluating your model on different subsets of the data, you account for variability due to different training/validation splits. This reduces the risk that your model's performance is overly dependent on a specific split of the data.
Improve Generalization: Averaging the results across multiple splits gives a better estimate of how your model is likely to perform on unseen data. It smooths out performance fluctuations due to random chance in how the data is divided.
Robustness to Outliers: Individual splits may be skewed by outliers or peculiarities in the data, but when averaged across multiple splits, these effects are minimized, resulting in a more robust estimate of model performance.
Techniques like k-fold cross-validation are commonly used to average performance across multiple data splits, providing a more accurate and stable evaluation of model generalization ability.
You said:
Does it give more accurate estimate of test accuracy?
ChatGPT said:
ChatGPT
Yes, averaging validation accuracy across multiple splits, as done in techniques like k-fold cross-validation, typically provides a more accurate estimate of test accuracy for several reasons:
1. Reduced Overfitting to a Single Split: A single train-test split can lead to an overestimation or underestimation of test accuracy, especially if the data split happens to be particularly favorable or unfavorable. By averaging across multiple splits, you avoid this bias and obtain a more balanced view of model performance.2. Improved Generalization: Since the model is evaluated on multiple validation sets (each corresponding to a different split), the average performance reflects the model’s ability to generalize across different subsets of the data, providing a more reliable estimate of how it will perform on unseen test data.3. Reduction of Variance: Single splits can produce high variance in accuracy estimates, particularly with small datasets or imbalanced data. By averaging across splits, you get a more stable and less noisy estimate of model performance, as the effect of random variations in individual splits is smoothed out.4. More Efficient Use of Data: In k-fold cross-validation, each data point is used both for training and validation, which makes the estimate of test accuracy more reliable, especially when the dataset is small.Overall Impact:
While cross-validation and averaging across splits do not guarantee perfect predictions of test accuracy, they generally give a more unbiased and reliable estimate than a single validation split. It reflects the model's expected performance on new, unseen data more accurately.
You said:
What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
ChatGPT said:
ChatGPT
Yes, increasing the number of iterations (or splits) in cross-validation can lead to a better estimate of test accuracy, but there are diminishing returns. Here's how the number of iterations affects the estimate:
1. Increased Stability and Lower Variance:More iterations, such as in k-fold cross-validation with a higher k, provide more data points for averaging, which generally reduces the variance in the accuracy estimate.As the number of iterations increases, the model gets evaluated on more diverse subsets of the data, which leads to a more stable and reliable estimate of the model’s generalization performance.For example, moving from a 5-fold cross-validation to a 10-fold cross-validation typically provides a slightly more stable estimate, though the improvement may not be dramatic.2. Better Use of Data:More iterations mean that each data point is more likely to be used in both training and validation, giving a more comprehensive assessment of the model.For example, in leave-one-out cross-validation (LOOCV), each data point is used in the validation set once, maximizing the use of the dataset for both training and validation. However, this approach can be computationally expensive, especially with large datasets, and can lead to high variance in individual validation results.3. Diminishing Returns:Beyond a certain point, increasing the number of iterations (e.g., moving from 10-fold to 20-fold cross-validation) will provide only marginal improvements in the accuracy estimate.The diminishing returns are due to the fact that as you increase the number of splits, each validation set gets smaller, which can slightly degrade the quality of each individual estimate. Additionally, the computational cost increases with more iterations.4. Trade-off with Computation Time:Higher iterations improve the reliability of the estimate, but they also come at the cost of increased computation time. For example, 10-fold cross-validation runs the model 10 times, which is computationally more expensive than 5-fold cross-validation. If computation is a limiting factor, there’s often a balance between accuracy of the estimate and computational efficiency.Key Considerations:Small datasets benefit more from higher iterations (e.g., more splits), as the variability between different splits is greater.For large datasets, even a modest number of splits like 5-fold or 10-fold can often provide a good estimate without needing more iterations.The ideal number of iterations depends on factors like dataset size, computational resources, and the desired balance between accuracy and speed.Summary:
Increasing the number of iterations tends to give a more reliable estimate of test accuracy, but after a certain point, the benefit diminishes while the computational cost increases. A moderate number of iterations, such as 5- or 10-fold cross-validation, often provides a good trade-off between accuracy and efficiency.
You said:
Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
ChatGPT said:
ChatGPT
Yes, increasing the number of iterations in cross-validation can help when dealing with a very small training or validation dataset, but there are important considerations and limitations.
How Increasing Iterations Helps with Small Datasets:
More Efficient Use of Data:
Cross-validation, especially k-fold cross-validation, ensures that every data point is used in both training and validation across different splits. This maximizes the use of the small dataset and provides a more comprehensive evaluation of the model.For instance, with 10-fold cross-validation, 90% of the data is used for training in each iteration, and each data point is used for validation exactly once. This is much more efficient than a simple train-test split, where only a fraction of the data would be used for validation.
Reduced Overfitting to a Single Split:
With a small dataset, a single train-test split might lead to overfitting or underfitting, depending on how the data is divided. Cross-validation, by averaging the results over multiple splits, reduces the chance that the model’s performance is overly dependent on one specific training or validation set.By increasing the number of iterations (i.e., reducing the size of the validation set in each split), you can mitigate this problem to some extent, as the model is evaluated across multiple, smaller validation sets.
Leave-One-Out Cross-Validation (LOOCV):
In leave-one-out cross-validation (LOOCV), each training set consists of all but one data point, with the remaining data point used as the validation set. This ensures that almost all available data is used for training in each iteration, which is useful when the dataset is extremely small.LOOCV maximizes the use of data but is computationally expensive and can lead to high variance in the validation results, since each validation set contains only one data point. However, it is an option when data is very limited.Limitations and Challenges:
Small Training Sets Still Pose a Problem:
Even with increased iterations or using LOOCV, the training set in each iteration may still be too small to capture the full complexity of the data, especially if the dataset has high variance or complex patterns.Small training sets can lead to high bias in the model, as it may not be able to learn effectively from limited information. This can result in underfitting, where the model fails to capture important patterns in the data.
High Variance in Results:
While more iterations (like LOOCV) reduce bias by utilizing the maximum amount of data for training, they can increase variance in the validation results. Each iteration’s performance might fluctuate significantly depending on the specific validation points used, especially when the validation set is very small.This makes the accuracy estimate less stable, even though it averages out over many iterations.
Computational Cost:
Increasing the number of iterations, particularly with LOOCV or very high k in k-fold cross-validation, can dramatically increase the computational cost, as the model needs to be trained and evaluated many times. This can become a bottleneck, especially for complex models or large datasets.Potential Solutions to Address Small Datasets:
Data Augmentation:
For very small datasets, especially in fields like image classification or NLP, data augmentation techniques can generate synthetic variations of the original data, effectively enlarging the dataset and improving model training. This is commonly used in deep learning but can be applied in other domains.
Transfer Learning:
In cases where data is very limited, using a pre-trained model (if applicable to your task) and fine-tuning it on the small dataset can greatly improve performance. This approach is common in areas like image processing or NLP, where large models trained on extensive datasets can be adapted to new tasks with small datasets.
Regularization:
Regularization techniques like L2 regularization or dropout (for neural networks) help prevent overfitting, which is a risk with small datasets. Regularization forces the model to generalize better to unseen data, even when the training data is limited.Summary:
Increasing iterations (e.g., k-fold cross-validation or LOOCV) can mitigate some of the issues related to small datasets by making better use of available data and reducing overfitting to a specific split. However, small training sets still limit the model's ability to learn complex patterns, and high variance in validation results can be an issue. Strategies like data augmentation, transfer learning, and regularization can further improve model performance when working with small datasets.


ChatGPT Auto
You said:
Does averaging the validation accuracy across multiple splits give more consistent results?
ChatGPT said:
ChatGPT
Yes, averaging the validation accuracy across multiple splits (e.g., in cross-validation) generally gives more consistent and reliable results. This approach helps to:
Reduce Variance: By evaluating your model on different subsets of the data, you account for variability due to different training/validation splits. This reduces the risk that your model's performance is overly dependent on a specific split of the data.
Improve Generalization: Averaging the results across multiple splits gives a better estimate of how your model is likely to perform on unseen data. It smooths out performance fluctuations due to random chance in how the data is divided.
Robustness to Outliers: Individual splits may be skewed by outliers or peculiarities in the data, but when averaged across multiple splits, these effects are minimized, resulting in a more robust estimate of model performance.
Techniques like k-fold cross-validation are commonly used to average performance across multiple data splits, providing a more accurate and stable evaluation of model generalization ability.
You said:
Does it give more accurate estimate of test accuracy?
ChatGPT said:
ChatGPT
Yes, averaging validation accuracy across multiple splits, as done in techniques like k-fold cross-validation, typically provides a more accurate estimate of test accuracy for several reasons:
1. Reduced Overfitting to a Single Split: A single train-test split can lead to an overestimation or underestimation of test accuracy, especially if the data split happens to be particularly favorable or unfavorable. By averaging across multiple splits, you avoid this bias and obtain a more balanced view of model performance.2. Improved Generalization: Since the model is evaluated on multiple validation sets (each corresponding to a different split), the average performance reflects the model’s ability to generalize across different subsets of the data, providing a more reliable estimate of how it will perform on unseen test data.3. Reduction of Variance: Single splits can produce high variance in accuracy estimates, particularly with small datasets or imbalanced data. By averaging across splits, you get a more stable and less noisy estimate of model performance, as the effect of random variations in individual splits is smoothed out.4. More Efficient Use of Data: In k-fold cross-validation, each data point is used both for training and validation, which makes the estimate of test accuracy more reliable, especially when the dataset is small.Overall Impact:
While cross-validation and averaging across splits do not guarantee perfect predictions of test accuracy, they generally give a more unbiased and reliable estimate than a single validation split. It reflects the model's expected performance on new, unseen data more accurately.
You said:
What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
ChatGPT said:
ChatGPT
Yes, increasing the number of iterations (or splits) in cross-validation can lead to a better estimate of test accuracy, but there are diminishing returns. Here's how the number of iterations affects the estimate:
1. Increased Stability and Lower Variance:More iterations, such as in k-fold cross-validation with a higher k, provide more data points for averaging, which generally reduces the variance in the accuracy estimate.As the number of iterations increases, the model gets evaluated on more diverse subsets of the data, which leads to a more stable and reliable estimate of the model’s generalization performance.For example, moving from a 5-fold cross-validation to a 10-fold cross-validation typically provides a slightly more stable estimate, though the improvement may not be dramatic.2. Better Use of Data:More iterations mean that each data point is more likely to be used in both training and validation, giving a more comprehensive assessment of the model.For example, in leave-one-out cross-validation (LOOCV), each data point is used in the validation set once, maximizing the use of the dataset for both training and validation. However, this approach can be computationally expensive, especially with large datasets, and can lead to high variance in individual validation results.3. Diminishing Returns:Beyond a certain point, increasing the number of iterations (e.g., moving from 10-fold to 20-fold cross-validation) will provide only marginal improvements in the accuracy estimate.The diminishing returns are due to the fact that as you increase the number of splits, each validation set gets smaller, which can slightly degrade the quality of each individual estimate. Additionally, the computational cost increases with more iterations.4. Trade-off with Computation Time:Higher iterations improve the reliability of the estimate, but they also come at the cost of increased computation time. For example, 10-fold cross-validation runs the model 10 times, which is computationally more expensive than 5-fold cross-validation. If computation is a limiting factor, there’s often a balance between accuracy of the estimate and computational efficiency.Key Considerations:Small datasets benefit more from higher iterations (e.g., more splits), as the variability between different splits is greater.For large datasets, even a modest number of splits like 5-fold or 10-fold can often provide a good estimate without needing more iterations.The ideal number of iterations depends on factors like dataset size, computational resources, and the desired balance between accuracy and speed.Summary:
Increasing the number of iterations tends to give a more reliable estimate of test accuracy, but after a certain point, the benefit diminishes while the computational cost increases. A moderate number of iterations, such as 5- or 10-fold cross-validation, often provides a good trade-off between accuracy and efficiency.


Yes, averaging validation accuracy across multiple splits, as done in techniques like k-fold cross-validation, typically provides a more accurate estimate of test accuracy for several reasons:
1. Reduced Overfitting to a Single Split: A single train-test split can lead to an overestimation or underestimation of test accuracy, especially if the data split happens to be particularly favorable or unfavorable. By averaging across multiple splits, you avoid this bias and obtain a more balanced view of model performance.2. Improved Generalization: Since the model is evaluated on multiple validation sets (each corresponding to a different split), the average performance reflects the model’s ability to generalize across different subsets of the data, providing a more reliable estimate of how it will perform on unseen test data.3. Reduction of Variance: Single splits can produce high variance in accuracy estimates, particularly with small datasets or imbalanced data. By averaging across splits, you get a more stable and less noisy estimate of model performance, as the effect of random variations in individual splits is smoothed out.4. More Efficient Use of Data: In k-fold cross-validation, each data point is used both for training and validation, which makes the estimate of test accuracy more reliable, especially when the dataset is small.Overall Impact:
While cross-validation and averaging across splits do not guarantee perfect predictions of test accuracy, they generally give a more unbiased and reliable estimate than a single validation split. It reflects the model's expected performance on new, unseen data more accurately.
4o


Yes, averaging the validation accuracy across multiple splits (e.g., in cross-validation) generally gives more consistent and reliable results. This approach helps to:
Reduce Variance: By evaluating your model on different subsets of the data, you account for variability due to different training/validation splits. This reduces the risk that your model's performance is overly dependent on a specific split of the data.
Improve Generalization: Averaging the results across multiple splits gives a better estimate of how your model is likely to perform on unseen data. It smooths out performance fluctuations due to random chance in how the data is divided.
Robustness to Outliers: Individual splits may be skewed by outliers or peculiarities in the data, but when averaged across multiple splits, these effects are minimized, resulting in a more robust estimate of model performance.
Techniques like k-fold cross-validation are commonly used to average performance across multiple data splits, providing a more accurate and stable evaluation of model generalization ability.
4o

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.